<a href="https://colab.research.google.com/github/vanderbilt-ml/51-prybol-mlproj-xclass/blob/main/project_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proposed Project

##Background
I work as a data science executive for a healthcare technology company that offers predictive analytics, data augumentation, and warehousing for health insurance companies, provider networks, health systems, and large employer groups. As part of our offerings we have developed a robust set of validations that allow us to standardize and normalize data at massive scales. Most of these validations however, are rules and/or statistics based and their development requires significant subject matter expert effort. As part of our continued push towards automation we have been seeking machine learning drive approachs to scale validations without similarly scaling SME effort. One potential validation would be to predict which diagnosis codes should appear on a medical claim based on the billed procedure codes and flag anomalous claims where the diagnosis codes do not match expectations. 


## Project Description
Predicting diagnosis codes from procedure codes appears to be a straightfoward proposition, but the processs is deceivingy complex. With the switch to ICD-10, there are now more than 70,000 unique procedure codes or "tokens". This number grows even larger with the inclusion of the Current Procedural Terminology (CPT) code set (10,471 unqiue CPT codes). For reference, this combined vocabulary is nearly three times larger than the vocabularly used in Google's seminal natural language processing paper on BERT (Bidirectional Encoder Representations from Transformers). Each healthcare encounter can result in any number of unique procedure codes being billed. While the number of codes billed in a single claim or encounter is technically unlimited, a typically range is anywhere from 1 to 25. Given the large vocabulary and the variable length of codes billed to a single encounter, one common way to handle these complexities is by treating medical codes as "words" in a "sentence" or "document". 

Addtionally, the model target, in this case the complete set of ICD diagnosis codes, contains more than 69,000 unique values. While some of these tokens are are quite common, others are exceedingly rare. Any loss function used during model training need to take into account the relative prevalences of these codes. It stands to reason that the more rare the code in the target set, the more imporant it may be to any anomaly detection engine. 

Extreme multi-label classification (XMC) is the problem of finding the relevant labels for an input, from a very large universe of possible labels. XMC is a natural fit for this project given the scope of the target. As a result, this project will borrow heavily from the fields of natural language processing and XMC.  

To date, no administrative claims data has been made publically available for research purposes. For this project I will leverage my existing access to the [MIMIC-III and MIMIC-IV (Medical Information Mart for Intensive Care)](https://mimic.mit.edu) research databases that combined, contain the data for over 100,000 intensive care stays. These databased has been augmented to include the billed procedure and diagnosis codes. While the total subset of unique codes will be limited by the types of nature of the dataset (limited to ICU stays), the overall scope should be representitive of the larger scale problem. 

# Performance Metrics
Peformance will be measured using metrics that have been widely adopted for XML and ranking tasks. Precision at $k$ (p$@k$) is one such metric that counts the fraction of correct predictions in the top $k$ scoring labels in $\hat{y}$, and has been widely utilized. The ranking measure Normalized (Discounted) Cumlative Gain at $k$ (nDCG$@k$) as another evaluation metric. The p$@k$ and nDCG$@k$ metrics are defined for a predicted score vector $\hat{\mathbf y} \in {\mathbb{R}}^{L}$ and ground truth label vector $\mathbf y \in \left\lbrace 0, 1 \right\rbrace^L$ as: 

\begin{align}
        \text{P}@k := \frac{1}{k} \sum_{l\in \text{rank}_k (\hat{\mathbf y})} \mathbf y_l\\[1em]
        \text{DCG}@k := \sum_{l\in {\text{rank}}_k (\hat{\mathbf y})} \frac{\mathbf y_l}{\log(l+1)}\\[1em]
        \text{nDCG}@k := \frac{{\text{DCG}}@k}{\sum_{l=1}^{\min(k, \|\mathbf y\|_0)} \frac{1}{\log(l+1)}},\\[1em]
    \end{align}

where, $\text{rank}_k(\mathbf y)$ returns the $k$ largest indices of $\mathbf{y}$ ranked in descending order.

For datasets that contain excessively popular labels (often referred to as "head" labels) such as diagnosis codes, high p$@k$ may be achieved by simply predicting head labels repeatedly irrespective of their relevance to the data point. To check for such trivial behavior, it is recommended that XC methods also be evaluated with respect to propensity-scored counterparts of the p$@k$ and nDCG$@k$ metrics (PSP$@k$ and PSnDCG$@k$) described below.

\begin{align}
        \text{PSP}@k := \frac{1}{k} \sum_{l\in \text{rank}_k (\hat{\mathbf y})} \frac{\mathbf y_l}{p_l}\\[1em]
        \text{PSDCG}@k := \sum_{l\in {\text{rank}}_k (\hat{\mathbf y})} \frac{\mathbf y_l}{p_l\log(l+1)}\\[1em]
        \text{PSnDCG}@k := \frac{{\text{PSDCG}}@k}{\sum_{l=1}^{k} \frac{1}{\log(l+1)}},\\[1em]
    \end{align}

where $p_l$ is the propensity score for label $l$ which helps in making metrics unbiased with respect to missing labels. Propensity-scored metrics place specific emphasis on performing well on tail labels and give reduced rewards for predicting popular or head labels. For this study we will use metrics$@k$ in $\{1,3,5\}$.

Given that as of the time of writing, no relevant paper on this topic exist, the overarching purpose of this project will be to estabish a benchmark from which future work can be compared against. 